In [12]:
import numpy as np
from copy import deepcopy

class NotValidMatrixError(Exception):
    print("Matrix is not valid")

class NoSolutionError(Exception):
    print("Matrix Has No solution")

class NoInverseError(Exception):
    print("Matrix Has No Inverse")


class SolveSPL(object):

    def __init__(self,matrix):
        self.origin = deepcopy(matrix)
        self.origin_np = np.mat(self.origin)
        self.matrix = matrix
        self.matrix_np = np.mat(matrix)
        self.row,self.collumn = self.matrix_np.shape
        self.solution=[None for i in range(self.row)]
        self.determinant = None
        
    def set_np(self):
        self.matrix_np = np.mat(self.matrix)
    
    def eliminate(self):
        row= self.row
        col=self.collumn
        for k in range(row-1):
            if abs(self.matrix[k][k]) < 10**-15:
                print("NotValidValue",(k,k))
                raise NotValidMatrixError
            for i in range(k+1,row):
                p = self.matrix[i][k]/self.matrix[k][k]
                for j in range(k+1,col):
                    self.matrix[i][j]= self.matrix[i][j] -p*self.matrix[k][j]
                self.matrix[i][k]=0
        self.set_np()
        
        return self.matrix
    
    def eliminate_parsial(self):
        row= self.row
        col=self.collumn
        
        for k in range(row-1):#Iterate from first row to the second last row
            
            #Begin pivoting
            m=k
            for i in range(k+1,row):#Iterate from next kth row to the last row
                
                if abs(self.matrix[i][k]) > abs(self.matrix[m][k]):
                    m=i
                    
            if m != k:
                for j in range(k,col):#Iterate from kth collumn to the last collumn
                    s=self.matrix[k][j]
                    self.matrix[k][j]=self.matrix[m][j]
                    self.matrix[m][j] =s
                
            if abs(self.matrix[k][k]) < 10**-15:
                print("NotValidValue",(k,k))
                raise NotValidMatrixError
                
            #Gauss Elimination    
            for i in range(k+1,row-1):#Iterate from first row to the second last row
                
                p=self.matrix[i][k]/self.matrix[k][k]
                
                for j in range(k+1,col):#Iterate from next to kth collumn to the last collumn
                    self.matrix[i][j]= self.matrix[i][j] -p*self.matrix[k][j]
                self.matrix[i][k]=0
        
        self.set_np()
        
        return self.matrix
    
    def backwards(self):
        row= self.row
        col=self.collumn
        dim= row -1
        if abs(self.matrix[dim][dim]) < 10**-15:
            print("NotValidValue",(dim,dim))
            raise NotValidMatrixError
        
        self.solution[dim] = self.matrix[dim][dim+1]/self.matrix[dim][dim]
        
        for k in range(dim-1,-1,-1):
            s=0
            for i in range(k+1,dim):
                s += self.matrix[k][i]*self.solution[i]
            
            self.solution[k]=self.matrix[k][dim+1]/self.matrix[k][k]
        
        if None in self.solution:
            raise NoSolutionError
        
        self.set_np()
        return self.matrix
    
    def find_determinant(self,pivot=False):
        row = self.row
        if self.matrix == self.origin :
            if pivot:
                self.eliminate()
            else:
                self.eliminate_parsial()
        print(np.array(self.matrix))
        det = 1
        for i in range(row):
            det = det*self.matrix[i][i]
        
        self.determinant = det
        
        return det
    
    def find_inverse(self,pivot=True):
        row=self.row
        col=self.collumn
        col2 = row*2
        inverse =[[0 for i in range(col2) ] for j in range(row)]
        for i in range(row):
            for j in range(col):
                inverse[i][j]= self.origin[i][j]
            for j in range(col-1,col2):
                if i==j-(col-1) :
                    inverse[i][j]=1
                else:
                    inverse[i][j]=0
        for k in range(row):
            m=k
            for i in range(k+1,col2):
                if abs(inverse[k][i]> abs(inverse[m][k])):
                    m=i
            if m != k:
                for j in range(k,col2):
                    s=inverse[k][j]
                    inverse[k][j]=inverse[m][j]
                    inverse[m][j]=s
            if abs(inverse[k][k]) < 10*-15:
                raise NoInverseError
                
                p=inverse[k][k]
                for j in range(k,col2):
                    inverse[k][j] = inverse[k][j]/p
                
                for i in range(row):
                    if i!=k:
                        p=inverse[i][k]
                        for j in range(k+1,col2):
                            inverse[i][j] =inverse[i][j] -p*inverse[k][j]
                        inverse[i][k]=0
        inverse= np.mat(inverse)
        return inverse
    
    
    def solve(self,pivot=False):
        
        if pivot:
            self.eliminate_parsial()
        else:
            self.eliminate()
        self.backwards()
        
        return self.solution   

Matrix is not valid
Matrix Has No solution
Matrix Has No Inverse


In [13]:
spl_1 = SolveSPL([[2,1,-2,3],[1,-1,-1,0],[1,1,3,12]])

In [14]:
print(spl_1.origin_np)

print(spl_1.find_determinant(False))

[[ 2  1 -2  3]
 [ 1 -1 -1  0]
 [ 1  1  3 12]]
[[ 2.   1.  -2.   3. ]
 [ 0.  -1.5  0.  -1.5]
 [ 1.   1.   3.  12. ]]
-9.0
